In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
import requests

url = "https://raw.githubusercontent.com/codewithharsha/ETL-Pipeline/main/Border_Crossing_Entry_Data.csv"
response = requests.get(url)

with open("Border_Crossing_Entry_Data.csv", "wb") as f:
    f.write(response.content)

print("Download complete.")


Download complete.


In [ ]:
data = pd.read_csv("Border_Crossing_Entry_Data.csv")

In [ ]:
data.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point
0,Jackman,Maine,104,US-Canada Border,Jan 2024,Trucks,6556,45.806,-70.397,POINT (-70.396722 45.805661)
1,Porthill,Idaho,3308,US-Canada Border,Apr 2024,Trucks,98,49.000,-116.499,POINT (-116.49925 48.999861)
2,San Luis,Arizona,2608,US-Mexico Border,Apr 2024,Buses,10,32.485,-114.782,POINT (-114.7822222 32.485)
3,Willow Creek,Montana,3325,US-Canada Border,Jan 2024,Pedestrians,2,49.000,-109.731,POINT (-109.731333 48.999972)
4,Warroad,Minnesota,3423,US-Canada Border,Jan 2024,Personal Vehicle Passengers,9266,48.999,-95.377,POINT (-95.376555 48.999)


In [ ]:
data.shape

(399406, 10)

#Handling Missing values

In [ ]:
data.isnull().sum()

,0
Port Name,0
State,0
Port Code,0
Border,0
Date,0
Measure,0
Value,0
Latitude,0
Longitude,0
Point,0


In [ ]:
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].fillna('Unknown')
    else:
        data[col] = data[col].fillna(0)

In [ ]:
data.describe()

,Port Code,Value,Latitude,Longitude
count,399406.000000,3.994060e+05,399406.000000,399406.000000
mean,2445.976593,2.894677e+04,43.967389,-99.719586
std,1204.285210,1.495028e+05,8.152762,18.258623
min,101.000000,0.000000e+00,25.952000,-141.001000
25%,2304.000000,0.000000e+00,42.999000,-114.728000
50%,3012.000000,1.470000e+02,48.122000,-101.628000
75%,3401.000000,3.029750e+03,49.000000,-84.361000
max,3814.000000,4.447374e+06,62.615000,-66.980000


# Handling Duplicates

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.shape

(399396, 10)

In [ ]:
#note there are 10 duplicate entries (399406-399396 = 10)

# Column Datatypes

In [ ]:
for col in data.columns:
    print(col,data[col].dtype)

Port Name object
State object
Port Code int64
Border object
Date object
Measure object
Value int64
Latitude float64
Longitude float64
Point object


In [ ]:
# Explicitly convert to string (if not already)
cols_to_convert = ['Port Name', 'State', 'Border', 'Measure']
for col in cols_to_convert:
    data[col] = data[col].astype(str)

In [ ]:
for col in data.columns:
    print(col,data[col].dtype)

Port Name object
State object
Port Code int64
Border object
Date object
Measure object
Value int64
Latitude float64
Longitude float64
Point object


#Processing point Column

In [ ]:
from shapely.wkt import loads
import geopandas as gpd

# Convert WKT Point string to shapely Point
data['geometry'] = data['Point'].apply(loads)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')


In [ ]:
for col in data.columns:
    print(col,data[col].dtype)

Port Name object
State object
Port Code int64
Border object
Date object
Measure object
Value int64
Latitude float64
Longitude float64
Point object
geometry object


#Processing date column

In [ ]:
# data['Date'] = pd.to_datetime(data['Date'], format='%b-%Y')
#the above code raises an error as all the columns aren't in the same format

In [ ]:
data['New Date'] = pd.to_datetime(data['Date'],errors='coerce')

<ipython-input-18-15cc754bd3d0>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['New Date'] = pd.to_datetime(data['Date'],errors='coerce')


In [ ]:
data.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point,geometry,New Date
0,Jackman,Maine,104,US-Canada Border,Jan 2024,Trucks,6556,45.806,-70.397,POINT (-70.396722 45.805661),POINT (-70.396722 45.805661),2024-01-01
1,Porthill,Idaho,3308,US-Canada Border,Apr 2024,Trucks,98,49.000,-116.499,POINT (-116.49925 48.999861),POINT (-116.49925 48.999861),2024-04-01
2,San Luis,Arizona,2608,US-Mexico Border,Apr 2024,Buses,10,32.485,-114.782,POINT (-114.7822222 32.485),POINT (-114.7822222 32.485),2024-04-01
3,Willow Creek,Montana,3325,US-Canada Border,Jan 2024,Pedestrians,2,49.000,-109.731,POINT (-109.731333 48.999972),POINT (-109.731333 48.999972),2024-01-01
4,Warroad,Minnesota,3423,US-Canada Border,Jan 2024,Personal Vehicle Passengers,9266,48.999,-95.377,POINT (-95.376555 48.999),POINT (-95.376555 48.999),2024-01-01


# Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[['Value_norm']] = scaler.fit_transform(data[['Value']])


# Aggregation

In [ ]:
state_summary = data.groupby('State')['Value'].sum().reset_index()
state_summary.rename(columns={'Value': 'Total Entries'}, inplace=True)

In [ ]:
state_summary

,State,Total Entries
0,Alaska,16477863
1,Arizona,1066059617
2,California,3061603336
3,Idaho,24551671
4,Maine,252803640
5,Michigan,849161399
6,Minnesota,108962610
7,Montana,62742402
8,New Mexico,86312805
9,New York,938773886


In [ ]:
monthly_avg = data.groupby(['Port Name', data['New Date'].dt.to_period('M')])['Value'].mean().reset_index()
monthly_avg['New Date'] = monthly_avg['New Date'].dt.to_timestamp()  # convert back to datetime

In [ ]:
monthly_avg

,Port Name,New Date,Value
0,Alcan,1996-01-01,284.666667
1,Alcan,1996-02-01,263.416667
2,Alcan,1996-03-01,400.583333
3,Alcan,1996-04-01,634.666667
4,Alcan,1996-05-01,1379.583333
...,...,...,...
37712,Ysleta,2024-10-01,183245.333333
37713,Ysleta,2024-11-01,176160.500000
37714,Ysleta,2024-12-01,205368.333333
37715,Ysleta,2025-01-01,186192.833333


In [ ]:
border_measure_summary = data.groupby(['Border', 'Measure'])['Value'].sum().reset_index()

In [ ]:
border_measure_summary

,Border,Measure,Value
0,US-Canada Border,Bus Passengers,78607720
1,US-Canada Border,Buses,3298832
2,US-Canada Border,Pedestrians,14216700
3,US-Canada Border,Personal Vehicle Passengers,1713361719
4,US-Canada Border,Personal Vehicles,827997558
5,US-Canada Border,Rail Containers Empty,16967752
6,US-Canada Border,Rail Containers Loaded,40387695
7,US-Canada Border,Train Passengers,6869760
8,US-Canada Border,Trains,822570
9,US-Canada Border,Truck Containers Empty,32526016


# Feature Engineering

In [ ]:
# Extract year, month, and day of week
data['Year'] = data['New Date'].dt.year
data['Month'] = data['New Date'].dt.month
data['Month_Name'] = data['New Date'].dt.month_name()

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

data['Season'] = data['Month'].apply(get_season)


In [ ]:
data['Traffic_Level'] = pd.cut(
    data['Value'],
    bins=[-1, 1000, 10000, float('inf')],
    labels=['Low', 'Medium', 'High']
)


In [ ]:
data.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point,geometry,New Date,Value_norm,Year,Month,Month_Name,Season,Traffic_Level
0,Jackman,Maine,104,US-Canada Border,Jan 2024,Trucks,6556,45.806,-70.397,POINT (-70.396722 45.805661),POINT (-70.396722 45.805661),2024-01-01,1.474128e-03,2024,1,January,Winter,Medium
1,Porthill,Idaho,3308,US-Canada Border,Apr 2024,Trucks,98,49.000,-116.499,POINT (-116.49925 48.999861),POINT (-116.49925 48.999861),2024-04-01,2.203548e-05,2024,4,April,Spring,Low
2,San Luis,Arizona,2608,US-Mexico Border,Apr 2024,Buses,10,32.485,-114.782,POINT (-114.7822222 32.485),POINT (-114.7822222 32.485),2024-04-01,2.248518e-06,2024,4,April,Spring,Low
3,Willow Creek,Montana,3325,US-Canada Border,Jan 2024,Pedestrians,2,49.000,-109.731,POINT (-109.731333 48.999972),POINT (-109.731333 48.999972),2024-01-01,4.497036e-07,2024,1,January,Winter,Low
4,Warroad,Minnesota,3423,US-Canada Border,Jan 2024,Personal Vehicle Passengers,9266,48.999,-95.377,POINT (-95.376555 48.999),POINT (-95.376555 48.999),2024-01-01,2.083477e-03,2024,1,January,Winter,Medium


In [ ]:
data.to_csv('processed_border_crossing_data.csv', index=False)